In [1]:
# !pip install -U imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.filters import sobel
from scipy.ndimage import gaussian_filter

from skimage.feature import canny
from skimage.filters import median
from skimage.restoration import denoise_bilateral

from skimage.filters import prewitt_h, prewitt_v

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

scaler = StandardScaler()

import joblib

In [2]:
# ref : https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/#gsc.tab=0

# DOWNSCALE BY 50% : (480,499) = size
def downsample_image(img):
    scale_percent = 20 
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized
 
def upsample_image(img):   # To be used after prediction
    scale_percent = 500 
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [3]:
### PREPROCESSING FUNCTION
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # check if size is correct 
    # 1) Convert to gray scale
    # 2) Normalize 
    # 3) Downsample image to 20%
    
    if(img.shape[0] != 960):
        print("Change shape")
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    final_img = gray_img/255.0
    final_img = downsample_image(gray_img)
    
    return final_img

def preprocess_seg_map(seg_path):
    # reading array 
    seg_map = cv2.imread(seg_path,cv2.IMREAD_GRAYSCALE)
    # Convert from bool to uint8
    seg_map = seg_map.astype(np.uint8)
    # convert the max value pixel to 1 (To create a binary seg map)
    seg_map[seg_map == 255] = 1
    
    # Downsample the seg_map by 50%
    seg_map = downsample_image(seg_map)
    return seg_map

def extract_features(gray_img,seg_map):
    
    radius = 1
    entropy_img = entropy(gray_img,disk(radius))
    sigma = 5  
    gauss_image = gaussian_filter(gray_img, sigma=sigma) #### (APPLIED ON COLOR ORIGINALLY)
    sobel_filtered = sobel(gray_img) 
    edges = canny(gray_img)
    median_filtered = median(gray_img)                               ##### (APPLIED ON COLOR ORIGINALLY)
    bilateral_filtered = denoise_bilateral(gray_img,channel_axis=-1,multichannel=False) ##### (APPLIED ON COLOR ORIGINALLY)
    prewitt_horizontal = prewitt_h(gray_img)
    prewitt_vertical = prewitt_v(gray_img)
    
    pixel_feature = gray_img.reshape(-1).reshape(-1, 1)                        #1
    entropy_feature = entropy_img.reshape(-1).reshape(-1, 1)                   #2
    gauss_feature = gauss_image.reshape(-1).reshape(-1, 1)                     #3
    sobel_feature = sobel_filtered.reshape(-1).reshape(-1, 1)                  #4
    edges_feature = edges.reshape(-1).reshape(-1, 1)                           #5
    median_feature = median_filtered.reshape(-1).reshape(-1, 1)                #6
    bilateral_feature = bilateral_filtered.reshape(-1).reshape(-1, 1)          #7
    prewitt_horizontal_feature = prewitt_horizontal.reshape(-1).reshape(-1, 1) #8
    prewitt_vertical_feature = prewitt_vertical.reshape(-1).reshape(-1, 1)     #9
    
    label = seg_map.reshape(-1).reshape(-1, 1) 
    
    feature_matrix = np.hstack((pixel_feature,entropy_feature,gauss_feature,
                               sobel_feature,edges_feature,median_feature,
                               bilateral_feature,prewitt_horizontal_feature,
                               prewitt_vertical_feature,label))
    return feature_matrix
    
def find_segfile_of_image(image,segmentations):
        image_name = image.split(".")[0]
        for seg in segmentations:
            if image_name in seg:
                return seg
            
#     print("entropy_feature : ",entropy_img.shape)
#     print("gauss_feature : ",gauss_image.shape)
#     print("sobel_feature : ",sobel_filtered.shape)
#     print("edges_feature : ",edges.shape)
#     print("median_feature : ",median_filtered.shape)
#     print("bilateral_feature : ",bilateral_filtered.shape)
#     print("prewitt_horizontal_feature : ",prewitt_horizontal.shape)
#     print("prewitt_vertical_feature : ",prewitt_vertical.shape)
    
#     print()
#     print("entropy_feature : ",entropy_feature.shape)
#     print("gauss_feature : ",gauss_feature.shape)
#     print("sobel_feature : ",sobel_feature.shape)
#     print("edges_feature : ",edges_feature.shape)
#     print("median_feature : ",median_feature.shape)
#     print("bilateral_feature : ",bilateral_feature.shape)
#     print("prewitt_horizontal_feature : ",prewitt_horizontal_feature.shape)
#     print("prewitt_vertical_feature : ",prewitt_vertical_feature.shape)

#### PREPROCESSING AND FEATURE EXTRACTION 

1) Preprocessing : 
    
    (a) Conversion to gray scale (For feature extraction)
    
    (b) Resizing to 150x150 if not training in 960x999 
    
    (c) Normalizing : divide by 255.0
    
2) Feature extraction

   Features:-
    (1) Entropy  (TEXTURE)
    
    (2) Sobel    
    
    (3) Gaussian          
    
    (4) Canny edge detector
    
    (5) Median filter
    
    (6) Bilateral filter
    
    (7) Prewitt vertical filter
    
    (8) Prewitt horizontal filter
    
    (9) Pixel intensity 
    
    CAN ADD GABOR FEATURES TO THIS AS WELL
    
    (10)Pixel label (1 or 0)
    
    PROBLEM : 1) Some features are found on gray scale some on color 
                 afterwards we get feature maps of different channels
                 How to flatten them to get consistent dimensions afterwards 
                 in order to get a feature vector for an image

In [4]:
image_folder = "IMAGES"
seg_folder = "SEG1"

images = os.listdir(image_folder)
segmentations = os.listdir(seg_folder)

image_features_matrices_combined = []

i = 0
for image in images:
    if(i == 3):
        break
    
    # preprocessing image
    image_path = os.path.join(image_folder,image)
    gray_img = preprocess_image(image_path)
    
    
    # preprocessing segmentation map
    seg_name = find_segfile_of_image(image,segmentations)
    seg_path = os.path.join(seg_folder,seg_name)
    seg_map = preprocess_seg_map(seg_path)
    
    
    # feature extraction    
    feature_matrix = extract_features(gray_img,seg_map)
    image_features_matrices_combined.append(feature_matrix)
    
    print("Feature matrix for image ",image," : ",feature_matrix.shape)
    i+=1
    
print()
Final_matrix = np.vstack(image_features_matrices_combined)
print("Final shape of data matrix : ",Final_matrix.shape)

#     print("dtype of image array : ",img.dtype)
#     print("dtype of seg map : ",seg_map.dtype)
#     dtype of image array :  uint8
#     dtype of seg map :  uint8

C:\Users\RITWIK KASHYAP\AppData\Local\Temp\ipykernel_41436\319280895.py:40: FutureWarning: `multichannel` is a deprecated argument name for `denoise_bilateral`. It will be removed in version 1.0. Please use `channel_axis` instead.
  bilateral_filtered = denoise_bilateral(gray_img,channel_axis=-1,multichannel=False) ##### (APPLIED ON COLOR ORIGINALLY)


Feature matrix for image  Image_01L.jpg  :  (38208, 10)
Feature matrix for image  Image_01R.jpg  :  (38208, 10)
Feature matrix for image  Image_02L.jpg  :  (38208, 10)

Final shape of data matrix :  (114624, 10)


#### TRAIN, VALIDATION AND TEST SPLITS 

In [5]:
df = pd.DataFrame(Final_matrix)
df

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
114619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
is_zero_df = df.eq(0).all().all()

if is_zero_df:
    print("The entire DataFrame is filled with 0's.")
else:
    print("The DataFrame contains non-zero values.")

The DataFrame contains non-zero values.


In [7]:
filtered_df = df[df.iloc[:, -1] == 1]
filtered_df

,0,1,2,3,4,5,6,7,8,9
900,28.0,2.321928,25.0,0.104576,1.0,28.0,0.155991,0.150327,-1.176471e-02,1.0
1082,45.0,2.321928,27.0,0.081467,0.0,42.0,0.177210,0.113725,-5.228758e-03,1.0
1083,42.0,2.321928,28.0,0.060380,0.0,42.0,0.181053,0.086275,2.222222e-02,1.0
1099,37.0,1.921928,28.0,0.030455,0.0,37.0,0.169538,0.044444,-1.045752e-02,1.0
1282,40.0,2.321928,31.0,0.001961,0.0,45.0,0.188996,0.001307,1.307190e-03,1.0
...,...,...,...,...,...,...,...,...,...,...
113134,58.0,1.921928,48.0,0.019460,0.0,60.0,0.210958,-0.005229,2.483660e-02,1.0
113146,66.0,1.370951,45.0,0.012401,0.0,66.0,0.170820,-0.009150,-1.568627e-02,1.0
113332,56.0,2.321928,43.0,0.003922,0.0,58.0,0.158567,-0.002614,2.614379e-03,1.0
113531,61.0,2.321928,38.0,0.068290,0.0,56.0,0.095545,-0.099346,6.535948e-03,1.0


#### Final_matrix : matrix that is to be used for training 
#### df : final df 

#### TRAIN : VALIDATION : TEST SPLIT ( 70 : 15 : 15)

In [8]:
# Random shuffle the matrix 
X = Final_matrix
np.random.shuffle(X)

n_samples = X.shape[0]

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

n_train = int(train_ratio * n_samples)
n_val = int(val_ratio * n_samples)
n_test = n_samples - n_train - n_val

X_train = X[:n_train, :]
X_val = X[n_train:n_train + n_val, :]
X_test = X[n_train + n_val:, :]

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (80236, 10)
X_val shape: (17193, 10)
X_test shape: (17195, 10)


#### CLASS IMBALANCE CHECK IN TRAIN MATRIX 

In [9]:
class_labels = X_train[:, -1]

count_class_0 = np.count_nonzero(class_labels == 0)
count_class_1 = np.count_nonzero(class_labels == 1)

print(f"Number of samples with class label 0: {count_class_0}")
print(f"Number of samples with class label 1: {count_class_1}")

Number of samples with class label 0: 74582
Number of samples with class label 1: 5654


In [10]:
X_train_features = X_train[:, :-1]
# X_train_features_scaled = scaler.fit_transform(X_train_features)

y_train_target = X_train[:, -1]

print("Before Xtrain shape : ",X_train_features.shape)
print("Before Ytrain shape : ",y_train_target.shape)
print()

count_zeros = np.count_nonzero(y_train_target == 0)
count_ones = np.count_nonzero(y_train_target == 1)
print("Number of zeros in ytrain before : ", count_zeros)
print("Number of ones ytrain before : ", count_ones)
print()
############################### CLASS BALANCING USING SMOTE #############################33
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_features, y_train_target)

############################### COUNTING CLASS WISE SAMPLES #############################33

count_zeros = np.count_nonzero(y_train_resampled == 0)
count_ones = np.count_nonzero(y_train_resampled == 1)

print("Number of zeros in ytrain after :", count_zeros)
print("Number of ones ytrain after :", count_ones)
print()
############################### SHAPES OF TRAINING DATA ####################################
print("X_train shape : ",X_train_resampled.shape)
print("y_train shape : ",y_train_resampled.shape)
print()

Before Xtrain shape :  (80236, 9)
Before Ytrain shape :  (80236,)

Number of zeros in ytrain before :  74582
Number of ones ytrain before :  5654

Number of zeros in ytrain after : 74582
Number of ones ytrain after : 74582

X_train shape :  (149164, 9)
y_train shape :  (149164,)



#### TRAINING :CLASS BALANCED

#### : 1) NORMAL LINEAR SVM

In [12]:
clf = svm.SVC(kernel='linear',C = 0.2,verbose = True,cache_size = 600)
print(clf)

SVC(C=0.2, cache_size=600, kernel='linear', verbose=True)


In [ ]:
clf.fit(X_train_resampled, y_train_resampled)

[LibSVM]

In [16]:
joblib.dump(clf, 'Trained_model_checkpoints/TM_4_301223_without_scaler_with_SMOTE_3Image.pkl')

['Trained_model_checkpoints/TM_2_271223_without_scaler_with_SMOTE_2Image.pkl']

#### CHECKING ACCURACY ON VALIDATION SET AFTER TRAINING MODEL

In [14]:
X_val_features = X_val[:, :-1]
# X_val_features_scaled = scaler.transform(X_val_features)

y_val_target = X_val[:, -1]

y_pred = clf.predict(X_val_features)

count_zeros = np.count_nonzero(y_pred == 0)
count_ones = np.count_nonzero(y_pred == 1)

print("Number of zeros:", count_zeros)
print("Number of ones:", count_ones)

accuracy = accuracy_score(y_val_target, y_pred)
report = classification_report(y_val_target, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Number of zeros: 4548
Number of ones: 1183
Accuracy: 0.8358052695864596
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.84      0.91      5361
         1.0       0.26      0.83      0.39       370

    accuracy                           0.84      5731
   macro avg       0.62      0.83      0.65      5731
weighted avg       0.94      0.84      0.87      5731



#### TEST SET ACCURACY 

In [15]:
X_test_features = X_test[:, :-1]
# X_test_features_scaled = scaler.transform(X_test_features)

y_test_target = X_test[:, -1]

y_pred = clf.predict(X_test_features)

accuracy = accuracy_score(y_test_target, y_pred)
report = classification_report(y_test_target, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.8354849965108164
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.83      0.90      5348
         1.0       0.27      0.85      0.41       384

    accuracy                           0.84      5732
   macro avg       0.63      0.84      0.66      5732
weighted avg       0.94      0.84      0.87      5732

